# 前処理大全 Python版

## はじめに
- 初めに以下のセルを実行してください
- 必要なライブラリのインポートとデータベース（PostgreSQL）からのデータ読み込みを行います
- pandas等、利用が想定されるライブラリは以下セルでインポートしています
- その他利用したいライブラリがあれば適宜インストールしてください（"!pip install ライブラリ名"でインストールも可能）
- 処理は複数回に分けても構いません
- 名前、住所等はダミーデータであり、実在するものではありません

In [2]:
import os
import pandas as pd
import numpy as np
from datetime import datetime, date
from dateutil.relativedelta import relativedelta
import math
import psycopg2
from sqlalchemy import create_engine
from sklearn import preprocessing
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import TimeSeriesSplit
from imblearn.under_sampling import RandomUnderSampler

production_tb = pd.read_csv("./data/production.csv")
monthly_index_tb = pd.read_csv("./data/monthly_index.csv")

# 演習問題

In [35]:
production_tb.head(5)

,type,length,thickness,fault_flg
0,E,274.027383,40.241131,False
1,D,86.319269,16.906715,False
2,E,123.940388,1.018462,False
3,B,175.554886,16.414924,False
4,B,244.934740,29.061081,False


In [34]:
monthly_index_tb.head(5)

,year_month,sales_amount,customer_number
0,2010-01,7191240,6885
1,2010-02,6253663,6824
2,2010-03,6868320,7834
3,2010-04,7147388,8552
4,2010-05,8755929,8171


---
> 005_split/01_train_test_split

In [4]:
X = production_tb.drop('fault_flg', axis=1)
y = production_tb[['fault_flg']]
train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.2)

train_X = train_X.reset_index(drop=True)
test_X  = test_X.reset_index(drop=True)
train_y = train_y.reset_index(drop=True)
test_y  = test_y.reset_index(drop=True)

- 知っておいた方がよいオプションとしては
  - seedで乱数を固定する
  - クラス分類の場合、stratifyにyを与えて、各クラスの数を分割前後で均等に維持する
  - あとshuffle=Trueにする。(デフォルトTrueだけど、KFoldがデフォルトFalseなので気持ち悪いんよね)
- その他分割時に考慮することとして
  - 全部の属性を混ぜた状態で分割しても大丈夫か？
  - 例えば、工場という属性が各データに含まれるとして、train/testに同じ工場が含まれても大丈夫か？
  - これはタスクのゴール・戦略による
    - 新しい工場に対しても頑健にしたいなら、trainとtestはお互い同じ工場を含まない方が良いはず。

In [5]:
X = production_tb.drop('fault_flg', axis=1)
y = production_tb[['fault_flg']]
train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.2, random_state=777, shuffle=True, stratify=y)

train_X = train_X.reset_index(drop=True)
test_X  = test_X.reset_index(drop=True)
train_y = train_y.reset_index(drop=True)
test_y  = test_y.reset_index(drop=True)

- あと、train_test_splitの引数は何個指定してもレコード数が同じであればいい感じに分けてくれる。

In [6]:
index = [*range(len(y))]

train_X, test_X, train_y, test_y, train_index, test_index = train_test_split(X, y, index, test_size=0.2)

train_X = train_X.reset_index(drop=True)
test_X  = test_X.reset_index(drop=True)
train_y = train_y.reset_index(drop=True)
test_y  = test_y.reset_index(drop=True)

In [7]:
test_y.query('not fault_flg')

,fault_flg
0,False
1,False
2,False
3,False
5,False
...,...
195,False
196,False
197,False
198,False


---
> 005_split/01_cross_valid

In [10]:
for train_cv_index, test_cv_index in KFold(n_splits=5, shuffle=True, random_state=777).split([*range(len(train_y))]):
    train_cv_X = train_X.iloc[train_cv_index, :]
    test_cv_X  = train_X.iloc[test_cv_index, :]
    
    train_cv_y = train_y.iloc[train_cv_index,:]
    test_cv_y  = train_y.iloc[test_cv_index,:]
    
    print(f"{len(train_cv_y)}({len(train_cv_y.query('fault_flg'))}:{len(train_cv_y.query('not fault_flg'))}), ", end="")
    print(f"{len(test_cv_y) }({len(test_cv_y.query('fault_flg')) }:{len(test_cv_y.query('not fault_flg')) })")

640(37:603), 160(5:155)
640(37:603), 160(5:155)
640(30:610), 160(12:148)
640(32:608), 160(10:150)
640(32:608), 160(10:150)


- 先ほどと同様にstratifiedなKFoldがある。

In [11]:
for train_cv_index, test_cv_index in StratifiedKFold(n_splits=5, shuffle=True, random_state=777).split(X=[*range(len(train_y))], y=train_y):
    train_cv_X = train_X.iloc[train_cv_index, :]
    test_cv_X  = train_X.iloc[test_cv_index, :]
    
    train_cv_y = train_y.iloc[train_cv_index,:]
    test_cv_y  = train_y.iloc[test_cv_index,:]
    
    print(f"{len(train_cv_y)}({len(train_cv_y.query('fault_flg'))}:{len(train_cv_y.query('not fault_flg'))}), ", end="")
    print(f"{len(test_cv_y) }({len(test_cv_y.query('fault_flg')) }:{len(test_cv_y.query('not fault_flg')) })")

640(34:606), 160(8:152)
640(34:606), 160(8:152)
640(34:606), 160(8:152)
640(33:607), 160(9:151)
640(33:607), 160(9:151)


---
> 005_split/02_time_series_split

- これはいろんな案がありそうですね。私は`range`を使いました。
- もっといいライブラリがあった気がするが、忘れてしまいました汗

In [75]:
TRAIN_WINDOW_LENGTH = 24
TRAIN_SHIFT_LENGTH = 12
TEST_WINDOW_LENGTH = 12
OFFSET = 0

for train_head in range(0, len(monthly_index_tb), SHIFT_LENGTH):
    train_head = train_head + OFFSET
    train_tail = train_head + TRAIN_WINDOW_LENGTH
    
    train_data = monthly_index_tb[train_head:train_tail]
    
    test_head = train_tail
    test_tail = test_head + TEST_WINDOW_LENGTH
    test_data = monthly_index_tb[test_head:test_tail]
    
    if len(train_data) < TRAIN_WINDOW_LENGTH \
        or len(test_data) < TEST_WINDOW_LENGTH:
        continue
    
    print(len(train_data), len(test_data))

24 12
24 12
24 12
24 12
24 12
24 12
24 12
24 12
